##個人課題##

##2522091 平 偉倫##
武蔵野大学のWebサイトのサイトマップを抽出する
武蔵野大学Webサイトのトップページにアクセス
同一ドメインの全てのリンク（コメントアウトされていないもの）を辿り，全ページのURLと<title>を辞書型変数に格納する
key：URL
value：<title></title>で挟まれた文字列
辞書型変数を print() で表示する

Webサイトにアクセスする際は必ず time.sleep(秒数) を使って負荷軽減をすること
課題用のipynb形式のファイルを作成し，githubフローでファイルの変更履歴を管理すること


In [ ]:
!python -m venv .venv
!source .venv/bin/activate  
!pip install requests beautifulsoup4 lxml

In [2]:
import time
import requests
from bs4 import BeautifulSoup

# 403回避
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

# 入口（トップページ）
BASE_URL = "https://www.musashino-u.ac.jp/"

In [ ]:

# リクエスト送信と基本情報表示
time.sleep(1)

# Web サーバへ GET リクエストを送信
res = requests.get(BASE_URL, headers=HEADERS)

# 文字化けを防ぐため、推定エンコーディングを設定
res.encoding = res.apparent_encoding

# レスポンスの基本情報を表示
print(f"レスポンス : {res}")                       
print(f"レスポンスの型 : {type(res)}")             
print(f"ステータスコード : {res.status_code}")    
print(f"リクエストURL : {res.request.url}")        
print(f"リクエストメソッド : {res.request.method}") 

レスポンス : <Response [200]>
レスポンスの型 : <class 'requests.models.Response'>
ステータスコード : 200
リクエストURL : https://www.musashino-u.ac.jp/
リクエストメソッド : GET


In [ ]:
# ヘッダ確認と本文の一部表示
print("レスポンスヘッダー :", res.headers)

# HTML テキストの先頭だけをプレビュー
print(res.text[:500])

レスポンスヘッダー : {'Content-Type': 'text/html', 'Content-Length': '18945', 'Connection': 'keep-alive', 'Date': 'Fri, 07 Nov 2025 10:04:12 GMT', 'Server': 'Apache', 'Last-Modified': 'Fri, 07 Nov 2025 08:03:13 GMT', 'Accept-Ranges': 'none', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 0960f8cb3feaf44b509547087ded384e.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'NRT57-P6', 'Alt-Svc': 'h3=":443"; ma=86400', 'X-Amz-Cf-Id': 'f3jWys5ED0_EO1t4VtuXqmk9Fta1p3z8ngcHOievyrlDzf_3u7naEw=='}
<!doctype html>
<html lang="ja">
	<head>

		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="viewport" id="viewport" content="width=device-width, initial-scale=1.0, user-scalable=yes">
		<meta name="format-detection" content="telephone=no">
		<meta name="theme-color" content="#541B86">

		<title>武蔵野大学</title>

		

In [ ]:
# ユーティリティ関数

import re
from urllib.parse import urljoin, urlparse, urldefrag

SKIP_EXTS = (".pdf", ".png", ".jpg", ".jpeg", ".gif", ".svg", ".zip", ".mp4", ".mp3", ".xlsx", ".docx", ".pptx")

# 同一ドメインの基準
TARGET_NETLOC = urlparse(BASE_URL).netloc  

def normalize_url(base, href):
    """# 絶対URL化 + フラグメント除去 + 細かな正規化"""
    if not href:
        return None
    href = href.strip()
    # javascript:, mailto:, tel: を除外
    if href.startswith(("javascript:", "mailto:", "tel:")):
        return None
    # 絶対URL化
    abs_url = urljoin(base, href)
    # フラグメント(#...)を除去
    abs_url, _ = urldefrag(abs_url)
    # 末尾の無意味なスラッシュ正規化
    return abs_url

def is_same_domain(url):
    """# 同一ドメイン判定"""
    try:
        return urlparse(url).netloc.endswith(TARGET_NETLOC)
    except Exception:
        return False

def is_skippable(url):
    """# バイナリ/大きいファイル拡張子などをスキップ"""
    lower = url.lower()
    return lower.endswith(SKIP_EXTS)

In [ ]:
#站内クロール（BFS）＋ <title> 収集

from collections import deque

# クロール設定
MAX_PAGES = 300   # 安全のため上限
DELAY_SEC = 1.0   # サーバー負荷軽減のための待機

visited = set()
queue = deque([BASE_URL])
url_title_dict = {} 

while queue and len(visited) < MAX_PAGES:
    url = queue.popleft()
    if url in visited:
        continue
    if not is_same_domain(url) or is_skippable(url):
        continue

    # アクセス前の待機
    time.sleep(DELAY_SEC)

    try:
        res = requests.get(url, headers=HEADERS, timeout=15)
    except requests.RequestException as e:
        # ネットワークエラー等はスキップ
        # print(f"アクセス失敗: {url} ({e})")
        continue

    # HTML以外はスキップ
    ctype = res.headers.get("Content-Type", "")
    if "text/html" not in ctype:
        continue

    # エンコーディング推定
    res.encoding = res.apparent_encoding
    html = res.text
    soup = BeautifulSoup(html, "lxml")

    # <title> の抽出
    title_tag = soup.find("title")
    title_text = title_tag.get_text(strip=True) if title_tag else ""

    # 成果物の辞書に保存
    url_title_dict[url] = title_text
    visited.add(url)

    # aタグから次リンクを収集
    for a in soup.find_all("a", href=True):
        nxt = normalize_url(url, a["href"])
        if not nxt:
            continue
        if not is_same_domain(nxt):
            continue
        if is_skippable(nxt):
            continue
        if nxt not in visited:
            queue.append(nxt)

    # 進捗の軽い表示
    if len(visited) % 20 == 0:
        print(f"進捗: {len(visited)}ページ収集 … 例: {url} -> {title_text[:30]}")

進捗: 20ページ収集 … 例: https://www.musashino-u.ac.jp/admission/graduate_school/ -> 大学院 入試情報 | 入試情報 | 武蔵野大学
進捗: 40ページ収集 … 例: https://www.musashino-u.ac.jp/academics/teachers_license/ -> 教職課程・国家資格 | 学部・大学院 | 武蔵野大学
進捗: 60ページ収集 … 例: https://www.musashino-u.ac.jp/student-life/campus_life/ -> キャンパスライフ | 学生生活・就職 | 武蔵野大学
進捗: 80ページ収集 … 例: https://www.musashino-u.ac.jp/happiness_creators/no027.html -> コンポストでキャンパスと地域の資源循環を実現 | 武蔵野大学
進捗: 100ページ収集 … 例: https://www.musashino-u.ac.jp/accesibility-and-copyright.html -> このサイトについて | 武蔵野大学
進捗: 120ページ収集 … 例: https://www.musashino-u.ac.jp/admission/faculty/result.html -> 入試結果 | 入試情報 | 武蔵野大学
進捗: 140ページ収集 … 例: https://www.musashino-u.ac.jp/student-life/career/qualification/qualification.html -> 目指せる資格・免許一覧 | 学生生活・就職 | 武蔵野大学
進捗: 160ページ収集 … 例: https://www.musashino-u.ac.jp/media.html -> 取材・ロケ撮影をお考えの方 | 武蔵野大学
進捗: 180ページ収集 … 例: https://www.musashino-u.ac.jp/guide/facility/physical_education_center.html -> 学院体育センター | 大学案内 | 武蔵野大学
進捗: 200ページ収集 … 例: https://www.musashino

In [7]:
#結果の確認と保存

print(f"収集URL数: {len(url_title_dict)}")

sample_n = 10
i = 0
for k, v in url_title_dict.items():
    print({k: v})
    i += 1
    if i >= sample_n:
        break

収集URL数: 300
{'https://www.musashino-u.ac.jp/': '武蔵野大学'}
{'https://www.musashino-u.ac.jp/access.html': '交通アクセス | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/admission/request.html': '資料請求 | 入試情報 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/contact.html': 'お問い合わせ | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/prospective-students.html': '武蔵野大学で学びたい方 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/students.html': '在学生の方 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/alumni.html': '卒業生の方 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/parents.html': '保護者の方 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/business.html': '企業・研究者の方 | 武蔵野大学'}
{'https://www.musashino-u.ac.jp/guide/': '大学案内 | 武蔵野大学'}


----------##自分で追加した機能##---------

In [10]:
from bs4 import BeautifulSoup
import requests

url = "https://www.musashino-u.ac.jp/"

res = requests.get(url)
res.encoding = res.apparent_encoding 

soup = BeautifulSoup(res.text, "html.parser")

In [ ]:
# 各ページで <meta name="description"> と <title> を取得
description_tag = soup.find("meta", attrs={"name": "description"})
description = description_tag["content"] if description_tag and "content" in description_tag.attrs else ""

# <title> タグからタイトルを取得
title_tag = soup.find("title")
title = title_tag.string if title_tag else ""

# URL, タイトル, 説明文を辞書型に格納
url_title_desc_dict = {}
url_title_desc_dict[url] = {
    "title": title,
    "description": description
}

# 出力して確認
print(url_title_desc_dict)

# 画像URLも追加
img_tags = soup.find_all("img", limit=3)
img_urls = [img.get("src") for img in img_tags if img.get("src")]

url_title_desc_dict[url] = {
    "title": title,
    "description": description,
    "images": img_urls
}

{'https://www.musashino-u.ac.jp/': {'title': '武蔵野大学', 'description': '武蔵野大学の公式サイトです。武蔵野大学は、世界の幸せをカタチにするために、学生、教職員、本学に関わりのあるすべての人々が感性、知恵、響創力を高め合うことを推進します。'}}


In [17]:
import csv

with open("musashino_sitemap_extended.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["URL", "TITLE", "DESCRIPTION", "IMAGES"])  

    for url, data in url_title_dict.items():
        
        if isinstance(data, dict):
            title = data.get("title", "")
            description = data.get("description", "")
            images = ", ".join(data.get("images", [])) 
            writer.writerow([url, title, description, images])
        else:
           
            writer.writerow([url, data, "", ""])